# Clone proyek Github untuk mendapatkan dataset

In [ ]:
# If run in google colab
!git clone https://github.com/ziszz/fake-news-classification.git

# Install library requirements

In [ ]:
# !pip install -r /content/fake-news-classification/requirements.txt

# Import library yang dibutuhkan

In [ ]:
import os
import pandas as pd
import tensorflow_model_analysis as tfma
from tfx.components import (CsvExampleGen, StatisticsGen, SchemaGen, 
                            ExampleValidator, Transform, Trainer, Tuner, 
                            Evaluator, Pusher)
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

# Set Variabel untuk directory pipeline

In [ ]:
PIPELINE_ROOT = "pipeline"
SCHEMA_PIPELINE_NAME = "fake-news-tfdv-schema"
SERVING_MODEL_DIR = "serving_model_dir"

# METADATA_PATH = os.path.join("metadata", PIPELINE_NAME, "metadata.db")

# Load Dataset

In [ ]:
# If run in google colab
real_df = pd.read_csv("/content/fake-news-classification/data/True.csv")

# If run in local
# real_df = pd.read_csv("data/True.csv")

real_df

In [ ]:
# If run in google colab
fake_df = pd.read_csv("/content/fake-news-classification/data/Fake.csv")

# If run in local
# fake_df = pd.read_csv("data/Fake.csv")

fake_df

# Menggabungkan Dataset
Karena dataset untuk fake news dan real news dipisah, maka harus dilakukan penggabungan terlebih dahulu sebelum data digunakan untuk processing dan pelatihan.

In [ ]:
data_path = "data"

real_df["class"] = 1
fake_df["class"] = 0

df = pd.concat([real_df, fake_df], axis=0)
df = df.drop(["title", "date", "subject"], axis=1)
df = df.rename(columns={"class": "is_real"})

# If run in google colab
if not os.path.exists(data_path):
    os.makedirs(data_path)

df.to_csv(os.path.join(data_path, "news.csv"), index=False)

# If run in local
# df.to_csv("data/csv/news.csv", index=False)

In [ ]:
# If run in google colab
DATA_ROOT = "data"

# If run in local
# DATA ROOT = "data/csv"

In [ ]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Load dataset menggunakan Example Gen
Load data yang telah digabungkan sebelumnya menggunakan Example Gen pada machine learning pipeline

In [ ]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=9),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=1),
    ])
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)
interactive_context.run(example_gen)

# Membuat summary statistic pada dataset

In [ ]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
interactive_context.run(statistics_gen)

In [ ]:
interactive_context.show(statistics_gen.outputs["statistics"])

# Membuat data schema

In [ ]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

In [ ]:
interactive_context.show(schema_gen.outputs["schema"])

# Membuat Validator
Melakukan validator dengan mendeteksi anomali pada data berdasarkan summary statistic dan schema pada data

In [ ]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"],
)
interactive_context.run(example_validator)

In [ ]:
interactive_context.show(example_validator.outputs["anomalies"])

# Preprocessing Data
Processing yang dilakukan hanya untuk merubah nama feature dan label yang telah di transform menjadi text_xf, is_real_xf, dan transform feature kedalam format lowercase string dan untuk casting label kedalam format int64 untuk memastikan tipe data pada label

In [ ]:
TRANSFORM_MODULE_FILE = "news_transform.py"

In [ ]:
%%writefile {TRANSFORM_MODULE_FILE}
import string
import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = "is_real"
FEATURE_KEY = "text"


def transformed_name(key):
    return f"{key}_xf"


def preprocessing_fn(inputs):
    outputs = dict()
    
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(
        inputs[FEATURE_KEY]
    )
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs

In [ ]:
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

# Tuner Hyperparameter
Melakukan tuning hyperparameter pada arsitektur model yang digunakan untuk mendapatkan hyperparameter terbaik.

In [ ]:
TUNER_MODULE_FILE = "news_tuner.py"

In [ ]:
%%writefile {TUNER_MODULE_FILE}
import keras_tuner as kt
import tensorflow as tf
import tensorflow_transform as tft
from typing import NamedTuple, Dict, Text, Any
from keras_tuner.engine import base_tuner
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs


LABEL_KEY = "is_real"
FEATURE_KEY = "text"
NUM_EPOCHS = 5

TunerFnResult = NamedTuple("TunerFnResult", [
    ("tuner", base_tuner.BaseTuner),
    ("fit_kwargs", Dict[Text, Any]),
])

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_binary_accuracy",
    mode="max",
    verbose=1,
    patience=10,
)


def transformed_name(key):
    return f"{key}_xf"


def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64):
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )
    
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY),
    )
    
    return dataset


def model_builder(hp, vectorizer_layer):
    num_hidden_layers = hp.Choice(
        "num_hidden_layers", values=[1, 2]
    )
    embed_dims = hp.Int(
        "embed_dims", min_value=16, max_value=128, step=32
    )
    lstm_units= hp.Int(
        "lstm_units", min_value=32, max_value=128, step=32
    )
    dense_units = hp.Int(
        "dense_units", min_value=32, max_value=256, step=32
    )
    dropout_rate = hp.Float(
        "dropout_rate", min_value=0.1, max_value=0.5, step=0.1
    )
    learning_rate = hp.Choice(
        "learning_rate", values=[1e-2, 1e-3, 1e-4]
    )

    inputs = tf.keras.Input(
        shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string
    )
    
    x = vectorizer_layer(inputs)
    x = layers.Embedding(input_dim=5000, output_dim=embed_dims)(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units))(x)

    for _ in range(num_hidden_layers):
        x = layers.Dense(dense_units, activation=tf.nn.relu)(x)
        x = layers.Dropout(dropout_rate)(x)
    
    outputs = layers.Dense(1, activation=tf.nn.sigmoid)(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=["binary_accuracy"],
    )
    
    return model


def tuner_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    train_set = input_fn(
        fn_args.train_files[0], tf_transform_output, NUM_EPOCHS
    )
    eval_set = input_fn(
        fn_args.eval_files[0], tf_transform_output, NUM_EPOCHS
    )

    vectorizer_dataset = train_set.map(
        lambda f, l: f[transformed_name(FEATURE_KEY)]
    )

    vectorizer_layer = layers.TextVectorization(
        max_tokens=5000,
        output_mode="int",
        output_sequence_length=500,
    )
    vectorizer_layer.adapt(vectorizer_dataset)
    
    tuner = kt.Hyperband(
        hypermodel=lambda hp: model_builder(hp, vectorizer_layer),
        objective=kt.Objective('binary_accuracy', direction='max'),
        max_epochs=NUM_EPOCHS,
        factor=3,
        directory=fn_args.working_dir,
        project_name="kt_hyperband",
    )
    
    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "callbacks": [early_stopping_callback],
            "x": train_set,
            "validation_data": eval_set,
            "steps_per_epoch": fn_args.train_steps,
            "validation_steps": fn_args.eval_steps,
        },
    )

In [ ]:
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=800),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=400),
)
interactive_context.run(tuner)

# Training Model
Melatih model dengan menggunakan hyperparameter yang telah dituning sebelumnya

In [ ]:
TRAINER_MODULE_FILE = "news_trainer.py"

In [ ]:
%%writefile {TRAINER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_hub as hub
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs


LABEL_KEY = "is_real"
FEATURE_KEY = "text"


def transformed_name(key): 
    return f"{key}_xf"


def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64):
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )
    
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY),
    )
    
    return dataset


def model_builder(vectorizer_layer, hp):
    inputs = tf.keras.Input(
        shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string
    )

    x = vectorizer_layer(inputs)
    x = layers.Embedding(input_dim=5000, output_dim=hp["embed_dims"])(x)
    x = layers.Bidirectional(layers.LSTM(hp["lstm_units"]))(x)

    for _ in range(hp["num_hidden_layers"]):
        x = layers.Dense(hp["dense_units"], activation=tf.nn.relu)(x)
        x = layers.Dropout(hp["dropout_rate"])(x)

    outputs = layers.Dense(1, activation=tf.nn.sigmoid)(x)
    
    model = tf.keras.Model(inputs=inputs, outputs = outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp["learning_rate"]),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()],
    )
    
    model.summary()
    
    return model


def _get_serve_tf_example_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(
            serialized_tf_examples, feature_spec
        )
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using transformed features
        return model(transformed_features)
    
    return serve_tf_examples_fn


def run_fn(fn_args: FnArgs):
    hp = fn_args.hyperparameters["values"]
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), "logs")
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq="batch"
    )
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor="val_binary_accuracy",
        mode="max",
        verbose=1,
        patience=10,
    )
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        monitor="val_binary_accuracy",
        mode="max",
        verbose=1,
        save_best_only=True,
    )
    callbacks = [
        tensorboard_callback, 
        early_stopping_callback, 
        model_checkpoint_callback
    ]
    
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    train_set = input_fn(
        fn_args.train_files, tf_transform_output, hp["tuner/epochs"]
    )
    eval_set = input_fn(
        fn_args.eval_files, tf_transform_output, hp["tuner/epochs"]
    )
    
    vectorizer_dataset = train_set.map(
        lambda f, l: f[transformed_name(FEATURE_KEY)]
    )
    
    vectorizer_layer = layers.TextVectorization(
        max_tokens=5000,
        output_mode="int",
        output_sequence_length=500,
    )
    vectorizer_layer.adapt(vectorizer_dataset)

    model = model_builder(vectorizer_layer, hp)
    
    model.fit(
        x=train_set,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_set,
        validation_steps=fn_args.eval_steps,
        callbacks=callbacks,
        epochs=hp["tuner/epochs"],
        verbose=1,
    )
    
    signatures = {
        "serving_default": _get_serve_tf_example_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(
                shape=[None],
                dtype=tf.string,
                name="examples",
            )
        )
    }
    
    model.save(
        fn_args.serving_model_dir, 
        save_format="tf", 
        signatures=signatures
    )

In [ ]:
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=800),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=400),
)
interactive_context.run(trainer)

# Analisis dan Evaluasi Model
Menganalisa dan mengevaluasi model dengan menggunakan beberapa metric beserta nilai threshold pada metric tersebut

In [ ]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing),
).with_id("Latest_blessed_model_resolver")

interactive_context.run(model_resolver)

In [ ]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key="is_real")],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name="ExampleCount"),
            tfma.MetricConfig(class_name="AUC"),
            tfma.MetricConfig(class_name="TruePositives"),
            tfma.MetricConfig(class_name="FalsePositives"),
            tfma.MetricConfig(class_name="TrueNegatives"),
            tfma.MetricConfig(class_name="FalseNegatives"),
            tfma.MetricConfig(class_name="BinaryAccuracy", 
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={"value": 0.6},
                    ),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={"value": 1e-4},
                    ),
                ),
            ),
        ])
    ]
)

In [ ]:
evaluator = Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    baseline_model=model_resolver.outputs["model"],
    eval_config=eval_config,
)
interactive_context.run(evaluator)

In [ ]:
eval_result = evaluator.outputs["evaluation"].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result)

# Export Model
Mengexport model menggunakan komponen pusher pada tensorflow extended agar dapat digunakan untuk proses deployment model.

In [ ]:
pusher = Pusher(
    model=trainer.outputs["model"],
    model_blessing=evaluator.outputs["blessing"],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=os.path.join(
                SERVING_MODEL_DIR, "fake-news-detection-model"
            ),
        )
    )
)

interactive_context.run(pusher)

# Compress folder pipelines dan serving_model

In [ ]:
!zip -r /content/zizz1181-pipeline.zip /content/pipeline

In [ ]:
!zip -r /content/serving_model_dir.zip /content/serving_model_dir/